In [1]:
import requests
import pandas as pd
import numpy as np

def getDataCentral(path, filename, ext = ".xls"):
    '''
    Funcion para descargar data del Banco central
    
    path: ruta del documento a descargar
    filename: nombre del archivo a grabar la data descargada
    ext: la extencion del archivo. Por lo general los del banco central son '.xls' 
    
    TODO: Falta poner que se descarguen en la carpeta de data
    
    '''
    resp = requests.get(path)
    
    with open(filename + ext, "wb") as output:
        output.write(resp.content)

def formatoData(nombre):
    if nombre == "IPC":
        nombres_ipc = ["periodo", "mes", "indice", "var_menusal", "var_dic","var_12_meses", "promedio"]
        ipc = pd.read_excel("IPC.xls", skiprows = 5, names = nombres_ipc, usecols = [0, 1, 2, 3, 4, 5, 6], skipfooter = 1)

        ipc = ipc.dropna(how = "all").head()

        ipc["periodo"] = ipc["periodo"].fillna(method="ffill")

        return ipc.to_csv("{}_clean.csv".format(nombre), index = False)
    
    elif nombre == "TPM":
        nombres_tpm = ["periodo", "Mes", "tasa_politica","facilidades_depositos", "facilidades_prestamo", "facilidades_lombarda"]
        
        tpm = pd.read_excel("TPM.xls",skiprows = 5, names = nombres_tpm, skipfooter = 1)

        tpm['periodo'] = tpm['periodo'].fillna(method='ffill')
        
        return tpm.to_csv("{}_clean.csv".format(nombre), index = False)
    
    elif nombre == "agregados_monetarios":
        
        nombre_agregados = ["ano","mes","billetes_monedas","depositos_transferibles"]
        
        agregados_monetarios = pd.read_excel("agregados_monetarios.xls", skiprows = 11, usecols = [0,1,2,3], names = nombre_agregados, skipfooter = 4)

        agregados_monetarios = agregados_monetarios.dropna(how = "all")
        
        return agregados_monetarios.to_csv("{}_clean.csv".format(nombre), index = False)
    
    elif nombre == "tasas_todas":
        
        tasa_cambio_todas = pd.read_excel("tasa_cambio_todas.xls", skiprows = 2)
        
        return tasa_cambio_todas.to_csv("{}_clean.csv".format(nombre), index = False)
    
    elif nombre == "tasa_dolar":
        
        tasa_dolar = pd.read_excel("tasa_dolar.xls", skiprows = 2)

        return tasa_dolar.to_csv("{}_clean.csv".format(nombre), index = False)
    
    elif nombre == "PIB":
        PIB = pd.read_excel("PIB.xls", skiprows = 5, skipfooter = 20, header = [1,2])

        PIB = PIB.dropna(how = "all").stack().reset_index().melt(id_vars = ["level_0", "level_1"])

        PIB.rename(columns={'level_0': 'componente', 'level_1': 'trimestre', 'COMPONENTES':'periodo'}, inplace=True)

        PIB.dropna(inplace = True)
        
        return PIB.to_csv("{}_clean.csv".format(nombre), index = False)


In [2]:
documentos = {"PIB": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-real/documents/pib_gasto_2007.xls",
              "IPC": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/precios/documents/ipc_base_2010.xls",
              "TPM": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-monetario-y-financiero/documents/Serie_TPM.xlsx",
              "tasa_dolar": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/mercado-cambiario//documents//TASA_DOLAR_REFERENCIA_MC.XLS",
              "tasa_cambio_todas": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/mercado-cambiario//documents//TASAS_CONVERTIBLES_OTRAS_MONEDAS.xls",
              "agregados_monetarios": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-monetario-y-financiero/documents/agregados_monetarios.xls",
              "reservas_internacionales": "https://gdc.bancentral.gov.do/Common/public/estad%C3%ADsticas/sector-externo/documents/reservas_internacionales.xls"}

In [3]:
for filename, path in documentos.items():
    getDataCentral(path, filename)

In [4]:
archivos = [ "IPC", "TPM", "tasa_dolar", "tasa_cambio_todas", "agregados_monetarios", "PIB"]

In [5]:
for i in archivos:
    formatoData(i)

In [163]:
df = pd.read_excel("reservas_internacionales.xls", skiprows = 6, skipfooter = 5, usecols = list(range(0,36))).dropna(how = 'all')

df = df.melt(id_vars = "MES")

df['variable'] = df['variable'].replace('Unnamed*', np.nan, regex = True).fillna(method='ffill')

df

,MES,variable,value
0,NaN,1985.0,BRUTAS
1,ENERO,1985.0,255
2,FEBRERO,1985.0,273
3,MARZO,1985.0,312.7
4,ABRIL,1985.0,320.3
5,MAYO,1985.0,232.3
6,JUNIO,1985.0,287.5
7,JULIO,1985.0,292.7
8,AGOSTO,1985.0,278
9,SEPTIEMBRE,1985.0,222.3


In [9]:
df = pd.read_excel("reservas_internacionales.xls", skiprows = 6, skipfooter = 5, usecols = 'A,AL:CG').dropna(how = 'all')

df = df.melt(id_vars = "MES")

df['variable'] = df['variable'].replace('Unnamed*', np.nan, regex = True).fillna(method='ffill')

df

,MES,variable,value
0,NaN,2003,ACTIVOS
1,NaN,2003,BRUTOS
2,ENERO,2003,784.5
3,FEBRERO,2003,875.5
4,MARZO,2003,842.6
5,ABRIL,2003,803.4
6,MAYO,2003,786.7
7,JUNIO,2003,716.4
8,JULIO,2003,583.1
9,AGOSTO,2003,508.7
